### Because the field of description looks noisy, we decides to conduct 3 fields generation instead

In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)

from utils.path import dir_HugeFiles
from utils.preprocessing import load
from sklearn.model_selection import train_test_split
from utils.save import make_dir, save_pickle, load_pickle, save


import pandas as pd
import numpy as np
import re

from utils.evaluation import spacy_extension
import tqdm

#dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
#dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.72 µs


In [75]:
dic = load(dir_save = '../big_data/dic_20190927.pickle')

ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist ../big_data/dic_20190927.pickle
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 4.7 s


In [3]:
ls = range(100)

time: 32.3 ms


In [3]:
### STEP3 sent to the NYtimes
### assign indices to each ingredient <---> NYtimes
class ny_ingredients:
    def __init__(self, fields):
        # this function will take the global variable ls and dic
        # static & reuseable
        self.ny_ingred = '../../NYtime-parser2/ingred.txt'
        self.ny_result = '../../NYtime-parser2/result.json'
        
        self.fields = fields #['ingredients', 'generated_ingred']
        # self.sp = spacy_extension() 

    def to_ny(self):
        '''
        using global variables data and ls
        '''
        to_write = []
        for i, v in tqdm.tqdm(enumerate(data)):
            # assing index
            for field in self.fields:
                line_ids = []
                for line in v[field]:
                    if line in to_write:
                        ny_id = to_write.index(line)
                    else:
                        ny_id = len(to_write)
                        to_write.append(line)
                    line_ids.append(ny_id)
                data[i]['ny_%s'%(field)] = line_ids

        # save the file to the folder under NYtime-parser2
        save(filename = self.ny_ingred, 
             to_write = '\n'.join(to_write),
             overwrite = True, 
             print_=True)

        self.to_write = to_write
        
    # step 3
    def to_ingred(self):
        '''
        using global variables dic and ls
        '''
        ny_result = pd.read_json(self.ny_result)
        to_write = []
        for i, v in tqdm.tqdm(enumerate(data)):
            if i in ls:
                # assing index
                for field in self.fields:
                    temp = [ny_result.loc[ny_id]['name'] for ny_id in v['ny_%s'%(field)]]
                    exact, root = self.extract(temp)
                    data[i]['ny_full_%s'%(field)] = ny_result.loc[ny_id].to_dict()
                    data[i]['ny_%s'%(field)] = {'ny':temp, 'exact':exact, 'root':root}
        # step 3
    def to_ingred2(self):
        '''
        using global variables dic and ls
        '''
        self.df_result = pd.read_json(self.ny_result)
        to_write = []
        for i, v in tqdm.tqdm(enumerate(data)):
            if i in ls:
                # assing index
                for field in self.fields:
                    data[i]['inv_%s'%(field)] = self.inverse_ingr(v['ny_%s'%(field)])
                    
    def inverse_ingr(self, x):
        '''
        x: something like data[0]['ny_ingredients']
        '''
        output = []
        for ny_id in x:   
            series = self.df_result.loc[ny_id]
            answer = []
            if type(series['name'])==str:
                answer.append(series['name'])
            if type(series['unit'])==str:
                answer.append(series['unit'])
            if type(series['qty'])==str:
                answer.append(series['qty'])
            # if type(series['name'])== float: # should change
            #    answer.append(series['input'])
            answer = ' '.join(answer)
            output.append(answer)
        return output
                    
    #def extract(self, ny_ingred):
    #    '''
    #    Args: ny_ingred: a list of ingredient names
    #    '''
    #    return self.sp.ny_ingred(ny_ingred)

time: 29.9 ms


In [7]:
data = [v for i, v in dic.items()]

time: 51.6 ms


In [9]:
### start                    
ny_ingr = ny_ingredients(fields = ['ingredients'])
### step 3-1 save it as ingred.txt
ny_ingr.to_ny()
### step 3-2 go to python2 and run NLG_notebooks/Control Nytimes

55102it [06:31, 140.84it/s]

saved ../../NYtime-parser2/ingred.txt
time: 6min 31s


In [10]:
### step 3-3 load the result.json back to dic
ny_ingr.to_ingred2()

55102it [03:03, 300.38it/s]

time: 3min 4s


In [76]:
save_pickle('../big_data/data55_inv.pickle', data, overwrite=True)

time: 2.17 s


In [16]:
def lowerTuples(aTup):
    result = ()
    for i in aTup:
        result += (i.lower(),) 
    return result

def int_to_roman(input):
    """ Convert an integer to a Roman numeral. """

    if not isinstance(input, type(1)):
        raise (TypeError, "expected integer, got %s" % type(input))
    if not 0 < input < 4000:
        raise (ValueError, "Argument must be between 1 and 3999")
    ints = (1000, 900,  500, 400, 100, 90, 50,  40, 10,  9,   5,  4,   1)
    nums = ('M',  'CM', 'D', 'CD','C', 'XC','L','XL','X','IX','V','IV','I')
    nums = lowerTuples(nums)
    result = []
    for i in range(len(ints)):
        count = int(input / ints[i])
        result.append(nums[i] * count)
        input -= ints[i] * count
    return ''.join(result)

romans = [int_to_roman(i) for i in range(1, 1000)]

time: 68.4 ms


In [30]:
def remove_number(name):
    '''
    name: data[6008]['name'][0]
    '''
    return ' '.join([w for w in name[0].split(' ') if w not in romans])

time: 28.9 ms


In [63]:
for i, v in enumerate(data):
    data[i]['title'] = [remove_number(data[i]['name'])]

time: 3.11 s


In [38]:
# check
for i in ls:
    if 'ii' in data[i]['name'][0] and i <50:
        print(data[i]['name'], data[i]['title'])

['chinese chicken salad iii'] ['chinese chicken salad']
['no bake peanut butter cookies ii'] ['no bake peanut butter cookies']
time: 59.6 ms


In [64]:
save_pickle('../big_data/data55_inv.pickle', data, overwrite=True)

time: 2.22 s


In [92]:
def reverse(x): return x
def txt(v, fields, mode = 'train'):
    '''
    fields: an order list, the last is the field to predict
    mode: test/train, return string X, y or X+y
    '''
    to_write = ''
    for field in fields:
        if field == 'title':
            name = [reverse(sent) for sent in v['title']]
            to_write += ' <start-title>'+' '.join(name)+' <end-title>'
        if field == 'ingredients':
            ingredients = [reverse(sent) for sent in v['inv_ingredients']]
            to_write += ' <start-ingredients>'+'$'.join(ingredients)+'$ <end-ingredients>'
        if field == 'directions':
            directions = [reverse(sent) for sent in v['directions']]
            to_write += ' <start-directions>'+' '.join(directions)+' <end-directions>'
                                                     
    if mode == 'train':
        return to_write
                                                     
    elif mode == 'test':
        field_to_predict = '<start-%s>'%fields[-1]
        to_X, to_y = to_write.split(field_to_predict)
        return to_X+field_to_predict, to_y

class to_gpt2:
    def __init__(self, data, ls):
        ls_train, self.ls_test, _, __ = train_test_split(ls, ls, 
                                                         test_size = 0.2, 
                                                         random_state = random_seed, 
                                                         shuffle = True)
        
        self.ls_train, self.ls_val, _, __ = train_test_split(ls_train, 
                                                             ls_train, 
                                                             test_size = 0.25, 
                                                             random_state = random_seed , 
                                                             shuffle = True)
        self.data = data
        
    def train(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                self.save(filename+'%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients']), overwrite)
                self.save(filename+'%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title']), overwrite)
                
    def train_reduce(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:      
                self.save(filename+'%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions']), overwrite)
                
    def test(self, ls, filename, overwrite = False, is_val = False):
        to_write = ''
        for i, v in enumerate(self.data):
            if i in ls:
                self.save(filename+'X/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[0], overwrite)
                self.save(filename+'X/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[0], overwrite)
                
                self.save(filename+'y/%d'%(i)+'d.txt', txt(v, ['title','ingredients','directions'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'i.txt', txt(v, ['title','directions','ingredients'], mode = 'test')[1], overwrite)
                self.save(filename+'y/%d'%(i)+'t.txt', txt(v, ['ingredients','directions','title'], mode = 'test')[1], overwrite)
                                
    def fast_export(self, filename, overwrite = False):
        prefix = 'sample_train/'
        self.train_reduce(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'sample_val/'
        self.train_reduce(self.ls_val[:50], filename+prefix,overwrite = overwrite)
        prefix = 'sample_test/'
        self.test(self.ls_test[:50], filename+prefix, overwrite = overwrite)
        
        prefix = 'train/'
        self.train_reduce(self.ls_train, filename+prefix, overwrite = overwrite)
        prefix = 'val/'
        self.train_reduce(self.ls_val, filename+prefix,overwrite = overwrite)
        prefix = 'test/'
        self.test(self.ls_test, filename+prefix, overwrite = overwrite)
        
        prefix = 'train_500/'
        self.test(self.ls_train[:500], filename+prefix, overwrite = overwrite)
        prefix = 'test_500/'
        self.test(self.ls_test[:500], filename+prefix, overwrite = overwrite)
        
    def save(self, filename, to_write, overwrite = False):
        make_dir(filename)
        if os.path.isfile(filename) == True and overwrite == False:
            print('already exists'+filename)
        else:    
            with open(filename,'w') as f:
                f.write('%s' % to_write)

time: 34.9 ms


In [93]:
model = to_gpt2(data, ls)

time: 104 ms


In [95]:
model.fast_export('../../to_gpt2/recipe54k_1008/',overwrite = True)

make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
make dir
time: 1min 15s


In [94]:
# rm -rf '../../to_gpt2/recipe54k_1008/'

time: 1.28 s


Options: <br>
1 Zip everything and use scp to transfer the files <br>
2 run on DGX server (faster)